In [1]:
import numpy as np
import SearchSpace as ss
import ModelBuild as Builder
import TrainModel as Trainer
from google.colab import drive
import pandas as pd
import os
import torch

import torch
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms

drive.mount('/content/drive')

Output hidden; open in https://colab.research.google.com to view.

In [2]:
random_params = ss.create_param_combinations(strategy="random")
print("random Search Combinations:", random_params)

random Search Combinations: [{'num_layers': 3, 'units_per_layer': 111, 'activation': 'sigmoid', 'learning_rate': 0.24280925771986567, 'batch_size': 64, 'dropout_rate': 0.4547337505534649, 'l2_reg_strength': 7.804823431023513e-06}]


In [3]:
# 定义超参数搜索函数
def random_search(train_set, val_set, epochs=10, save_dir="/content/drive/MyDrive/DL_HPO/RandomResult"):
    all_results = []
    best_result = None
    best_val_acc = 0.0
    best_model_wts = None
    total_training_time = 0  # 初始化总训练时间

    # 确保保存路径存在
    os.makedirs(save_dir, exist_ok=True)

    # 检查是否已经存在汇总文件，如果存在，直接加载以继续
    summary_file_path = f"{save_dir}/random_result.csv"
    if os.path.exists(summary_file_path):
        existing_results_df = pd.read_csv(summary_file_path)
        total_training_time = existing_results_df["total_training_time"].max()  # 获取已保存的总时间

    # 遍历超参数组合，使用指定的起始索引
    for param_index in range(1000):

        params = ss.create_param_combinations(strategy="random")# 获取当前超参数组合
        param = params[0]
        print(param)
        # 使用当前超参数组合构建模型

        model = Builder.build_model(param)

        # 创建 DataLoader
        #print(type(param["batch_size"]))
        batch_size = int(param["batch_size"])
        train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

        # 训练模型
        result = Trainer.train_model(model, train_loader, val_loader,epochs=epochs)

        # 累加当前模型的训练时间到总时间
        total_training_time += result["training_time"]

        # 将当前超参数组合和结果合并
        result_summary = {
            "param_index": param_index,
            "params": params,
            "strategy": "random",  # 替换为当前使用的策略名称
            "train_loss": result["train_losses"][-1],
            "val_loss": result["val_losses"][-1],
            "train_accuracy": result["train_accuracies"][-1],
            "val_accuracy": result["val_accuracies"][-1],
            "training_time": result["training_time"],
            "total_training_time": total_training_time  # 记录总训练时间
        }

        all_results.append(result_summary)

        # 追加当前结果到汇总文件
        result_df = pd.DataFrame([result_summary])
        if os.path.exists(summary_file_path):
            result_df.to_csv(summary_file_path, mode='a', header=False, index=False)
        else:
            result_df.to_csv(summary_file_path, mode='w', index=False)

        # 更新最佳模型并保存
        if result["val_accuracies"][-1] > best_val_acc:
            best_val_acc = result["val_accuracies"][-1]
            best_result = result_summary
            best_model_wts = model.state_dict()  # 保存最佳模型权重

            # 保存最佳结果到 Google Drive
            best_result_df = pd.DataFrame([best_result])
            best_result_df.to_csv(f"{save_dir}/best_random_result.csv", index=False)
            torch.save(best_model_wts, f"{save_dir}/best_model_weights.pt")

    return all_results, best_result, best_model_wts


In [4]:
# 下载并预处理 MNIST 数据集
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# 加载 MNIST 数据集
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# 截取 1000 个训练样本和 100 个验证样本
train_subset = Subset(train_dataset, range(3000))
val_subset = Subset(test_dataset, range(500))

In [ ]:
all_results, best_result, best_model_wts = random_search(train_subset, val_subset, epochs=10,save_dir="/content/drive/MyDrive/DL_HPO/RandomResult")

{'num_layers': 2, 'units_per_layer': 20, 'activation': 'sigmoid', 'learning_rate': 0.023270553147617893, 'batch_size': 32, 'dropout_rate': 0.5339275465519897, 'l2_reg_strength': 0.00010350702887318447}
{'num_layers': 3, 'units_per_layer': 17, 'activation': 'relu', 'learning_rate': 0.01626696724901744, 'batch_size': 64, 'dropout_rate': 0.5798931656157149, 'l2_reg_strength': 0.00012547531908878502}
{'num_layers': 4, 'units_per_layer': 66, 'activation': 'sigmoid', 'learning_rate': 0.054307606090745725, 'batch_size': 256, 'dropout_rate': 0.35432878082316527, 'l2_reg_strength': 0.0004300769413823904}
{'num_layers': 3, 'units_per_layer': 120, 'activation': 'sigmoid', 'learning_rate': 0.00012725192621906533, 'batch_size': 64, 'dropout_rate': 0.06961473147590176, 'l2_reg_strength': 8.013351158461121e-07}
{'num_layers': 4, 'units_per_layer': 113, 'activation': 'relu', 'learning_rate': 0.0045196023954169155, 'batch_size': 64, 'dropout_rate': 0.3261147328997915, 'l2_reg_strength': 5.4076462125706